In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasetfornina/ninaprodb1train.pkl
/kaggle/input/datasetfornina/ninaprodb1test.pkl


In [2]:
import numpy as np
import os
import random
#from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import math

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras import layers, Sequential, optimizers, Input, Model

**WORKED CODE HERE**

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import gc
import random
import os

import matplotlib.pyplot as plt
import json

In [6]:
epochs = 64
learning_rate = 1e-3
batch_size = 16
method = "default"
dataset_type = 1

In [7]:
pip install PyWavelets


Note: you may need to restart the kernel to use updated packages.


In [8]:
### import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import pywt
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical


class Nina1Dataset(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.scaler = StandardScaler()
        self.scaler.fit(np.concatenate(self.dataframe['emg'].tolist()))
        
    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, idx):
        batch_data = self.dataframe[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_input_data = []
        batch_labels = []

        for i, target_row in batch_data.iterrows():
            data = target_row['emg'][:500]

            # Zero-Padding
            if len(data) < 500:
                data = np.concatenate((data, np.zeros((500 - len(data), 10))), axis=0)
            coeffs = pywt.wavedec(data, 'sym8')  # Using 3 decomposition levels as an example
            coeffs[1:] = (pywt.threshold(c, value=0.5) for c in coeffs[1:])
            data = pywt.waverec(coeffs, 'sym8')
            data = self.scaler.transform(data)
            
            # Division data by time-segment
            #input_data = np.transpose(data.reshape((25, 20, 10)), (0, 2, 1))
            input_data = data.reshape((25, 20, 10))
            label = target_row['stimulus']
            #label = to_categorical(label)
            batch_input_data.append(input_data)
            batch_labels.append(label)

        # Check if the batch size is smaller than the desired batch_size
        if len(batch_data) < self.batch_size:
            # Create a dummy batch with all elements set to zero
            dummy_input_data = np.zeros((self.batch_size,) + input_data.shape, dtype=np.float32)
            dummy_labels = np.zeros((self.batch_size,), dtype=np.int32)
            dummy_input_data[:len(batch_input_data)] = np.array(batch_input_data)
            dummy_labels[:len(batch_labels)] = np.array(batch_labels)
            dummy_labels = to_categorical(dummy_labels,num_classes=52)  # Convert labels to one-hot encoding
            
            return dummy_input_data, dummy_labels
        
        batch_labels = to_categorical(batch_labels,num_classes=52)  # Convert labels to one-hot encoding
        
        return np.array(batch_input_data), np.array(batch_labels)





# Parameters
batch_size = 32
train_dir = '/kaggle/input/datasetfornina/ninaprodb1train.pkl'
test_dir = '/kaggle/input/datasetfornina/ninaprodb1test.pkl'

# Set up dataset

train = pd.read_pickle(train_dir)
eval_data = pd.read_pickle(test_dir)

# Load train data
train_data = pd.read_pickle(train_dir)

# Split data into train and validation sets
#train_data, val_data = train_test_split(train_data, test_size=0.3, random_state=21)

train_data = train_data.sample(frac=1).reset_index(drop=True)
#val_data = val_data.sample(frac=1).reset_index(drop=True)
eval_data = eval_data.sample(frac=1).reset_index(drop=True)

# Create train, test and validation datasets
train_dataset = Nina1Dataset(train_data, batch_size=batch_size)
#val_dataset= Nina1Dataset(val_data, batch_size=batch_size)
test_dataset= Nina1Dataset(eval_data, batch_size=batch_size)


print(train_dataset)

In [9]:
import tensorflow as tf

from tensorflow.keras.activations import tanh
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape

from tensorflow.keras.layers import LSTM, Bidirectional, Dropout
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dropout, Input, Conv1D, MaxPooling1D,
    Flatten, Dense, ReLU, TimeDistributed, Reshape, BatchNormalization
)
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras import regularizers, initializers
def cnn(x):
    #print(x.shape)
    #x = Reshape((25, 20,10))(x)  
    print(x.shape)
    x = TimeDistributed((Conv1D(64, kernel_size=9, strides=2, padding='same', activation=tanh)))(x)
    print(x.shape)
    #(Batch, 10, 64)
    #x = TimeDistributed(Dropout(0.2093))(x)
    #x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    x = TimeDistributed(MaxPooling1D(pool_size=8, strides=2))(x)
    print(x.shape)
    #(Batch, 2, 64)
    
    #x = TimeDistributed(Dropout(0.2093))(x)
    
    
    x = TimeDistributed((Conv1D(64, kernel_size=5, strides=2, padding='same', activation=tanh)))(x)
    #(Batch, 1, 64)
    x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
    
    
    x = TimeDistributed((Conv1D(64, kernel_size=5, strides=2, padding='same', activation=tanh)))(x)
    print(x.shape)
    #(Batch, 1, 64)
    x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
   
    
    x = TimeDistributed((Conv1D(64, kernel_size=3, strides=2, padding='same', activation=tanh)))(x)
    #print(x.shape)
    #(Batch, 1, 64)
    x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    #x = TimeDistributed(BatchNormalization(epsilon=1e-6, momentum=0.95))(x)
    x = TimeDistributed(Flatten())(x)
    print(x.shape)
    # (Batch, 64)

    return x

def Bi_LSTMModel(input_shape,x):
    #model = Sequential()
    # Hidden dimensions
    hidden_dim = 200
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True, dropout=0.3), input_shape=input_shape)(x)
    #x = Dropout(0.2093)(x)
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True, dropout=0.3))(x)

    print(x.shape)
    print(11)
    #x = Dropout(0.2093)(x)
    x = Flatten()(x)
    # ( ,10000)

    return x
def EMGHandNet(input_shape, num_classes):
    # Define the input layer
    x = Input(shape=input_shape)
    inputs = x
    #print(x.shape)
    #(batch, 25, 20, 10)
    #temp = [cnn(x[:, t, :, :]) for t in range(x.shape[1])]
    #x = tf.stack(temp, axis=1)
    #print(x.shape)
    x = cnn(x)
   
    #print(x.shape)
    x = Bi_LSTMModel(x.shape[1:],x)
    #print(x.shape)
    #x = Dropout(0.2093)(x)
    
    x = Dense(512, activation='tanh')(x)
    #print(x.shape)
    x = Dropout(0.2093)(x)
    #x = BatchNormalization(epsilon=1e-6, momentum=0.95)(x)

    # Add the output layer
    output_layer = Dense(52, activation='softmax')(x)
    # Create the model
    model = Model(inputs=inputs, outputs=output_layer)

    return model



num_classes = 52  # Adjust based on the number of hand activity classes


# Create an instance of the EMGHandNet model
model = EMGHandNet((25,20,10), 52)


# Compile the model


initial_learning_rate = 0.001
decay_steps = 200
decay_rate = 0.9
batch_size =32
# Define your model and its optimizer
# Define learning rate schedule
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

    # Compile the model with learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
with tf.device('/GPU:0'):
    history = model.fit(train_dataset,
                        epochs=200,
                        batch_size=32,
                        validation_data=test_dataset
                        )

(None, 25, 20, 10)
(None, 25, 10, 64)
(None, 25, 2, 64)
(None, 25, 1, 64)
(None, 25, 64)
(None, 25, 64)
11
(None, 25, 400)
11
(None, 25, 400)
11
Epoch 1/200
308/308 [==============================] - 35s 46ms/step - loss: 3.1080 - accuracy: 0.2120 - val_loss: 2.2665 - val_accuracy: 0.3594
Epoch 2/200
308/308 [==============================] - 13s 41ms/step - loss: 2.2475 - accuracy: 0.3718 - val_loss: 1.5509 - val_accuracy: 0.5331
Epoch 3/200
308/308 [==============================] - 12s 39ms/step - loss: 1.7731 - accuracy: 0.4825 - val_loss: 1.2601 - val_accuracy: 0.6179
Epoch 4/200
308/308 [==============================] - 13s 41ms/step - loss: 1.4272 - accuracy: 0.5658 - val_loss: 0.9688 - val_accuracy: 0.6989
Epoch 5/200
308/308 [==============================] - 12s 39ms/step - loss: 1.1338 - accuracy: 0.6476 - val_loss: 0.8247 - val_accuracy: 0.7495
Epoch 6/200
308/308 [==============================] - 13s 41ms/step - loss: 0.9171 - accuracy: 0.7126 - val_loss: 0.6292 - val_ac

In [10]:
true_labels = []  # True labels for the test data
predicted_labels = []  # Predicted labels for the test data

for batch_data, batch_labels in test_dataset:
    batch_predictions = model.predict(batch_data)
    batch_predicted_labels = np.argmax(batch_predictions, axis=1)
    predicted_labels.extend(batch_predicted_labels)
    true_labels.extend(np.argmax(batch_labels, axis=1))

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

accuracy = np.mean(true_labels == predicted_labels)

print(accuracy)

1/1 [==============================] - 0s 25ms/step
0.9190340909090909
